# 데이터 : CMU-SE Dataset
# 현재 Mode Collapse 문제 발생

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gensim
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Input, Dense, Lambda
from keras.models import Model, Sequential
from keras import backend as K
from keras import objectives
from keras.activations import softmax
from keras.objectives import binary_crossentropy as bce
import tensorflow as tf
from tensorflow.contrib.distributions import RelaxedOneHotCategorical as gumbel

c:\users\notebook2\anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


## Read Data

In [2]:
raw_data = pd.read_csv("./train.txt",sep="\n")

## ingore length <7 sentence,put it in length=7, split sentence 0:7 which length >7

In [3]:
data = []
for i in raw_data["text"]:
    split_text = i.split(" ")
    if len(split_text) <7:
        pass
    elif len(split_text) == 7:
        data.append(split_text)
    else:
        data.append(split_text[0:7])

## make word2vec model, dimension = 64

In [4]:
model = gensim.models.Word2Vec(data,size = 64)

## change sentence to vector stack

In [5]:
sentence_to_word_vec = np.zeros(shape=(len(data),7,64))
for sentence_index, i in enumerate(data):
    temp_list = np.zeros(shape=(7,64))
    for idx, j in enumerate(i):
        try:
            temp_list[idx] = np.array([model.wv.get_vector(j)])
        except:
            temp_list[idx] = np.array([model.wv.get_vector("<unk>")])
    temp_list = np.reshape(temp_list,(1,7,64))
    sentence_to_word_vec[sentence_index] = temp_list

In [6]:
sentence_to_word_vec.shape

(39812, 7, 64)

## add one dimension 

In [7]:
sentence_to_word_vec = np.expand_dims(sentence_to_word_vec,axis = -1)

## Gumbel distribution

In [8]:
def gumbel_loss(x, x_hat):
    q_y = K.reshape(logits_y, (-1, N, M))
    q_y = softmax(q_y)
    log_q_y = K.log(q_y + 1e-20)
    kl_tmp = q_y * (log_q_y - K.log(1.0/M))
    KL = K.sum(kl_tmp, axis=(1, 2))
    elbo = data_dim * bce(x, x_hat) - KL 
    return elbo

## Make GAN(based on DCGAN, infoGAN)

In [9]:
class GAN2vec():
    def __init__(self):
        # Input shape
        self.sentence_length = 7
        self.word_dimension = 64
        self.channels = 1
        self.sentence_shape = (self.sentence_length, self.word_dimension, self.channels)
        self.latent_dim = 100

        optimizer = Adam(lr=0.0001, beta_1=0.5, beta_2=0.999)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        sentence = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(sentence)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Reshape((1,1,512)))
        model.add(Conv2DTranspose(256,kernel_size=(3,16),strides=2))
        model.add(Activation("relu"))
        model.add(Conv2DTranspose(1,kernel_size=(3,34),strides=2))
        model.add(Reshape((7,64,1)))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        sentence = model(noise)

        return Model(noise, sentence)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(256, kernel_size=(3,64), input_shape=self.sentence_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(128, kernel_size=(5,1)))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        sentence = Input(shape=self.sentence_shape)
        validity = model(sentence)

        return Model(sentence, validity)
    
    
    def pretrain_D(self, epochs, batch_size = 128):
        X_train = sentence_to_word_vec
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        valid = valid*0.9
        fake = fake+0.1
        print("pretraining D")
        for epoch in range(epochs):
            print("{}epochs".format(epoch))
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            sentences = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_sentences = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(sentences, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_sentences, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = sentence_to_word_vec


        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        valid = valid*0.9
        fake = fake+0.1
        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.show_sentence(epoch)

    def show_sentence(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_sentence = GAN2vec.generator.predict(noise)
        test = np.squeeze(gen_sentence)
        for i in test:
            sentence = ""
            for j in i:
                temp = model.wv.similar_by_vector(j)
                sentence=sentence+temp[0][0]+" "
            print(sentence)
            
    def predict(self):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_sentence = GAN2vec.generator.predict(noise)
        test = np.squeeze(gen_sentence)
        sentence_list = []
        for i in test:
            sentence = ""
            for j in i:
                temp = model.wv.similar_by_vector(j)
                sentence=sentence+temp[0][0]+" "
            sentence_list.append(sentence)
        return sentence

In [62]:
GAN2vec = GAN2vec()
GAN2vec.pretrain_D(epochs = 100)
GAN2vec.train(epochs=4000, batch_size=32, save_interval=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 5, 1, 256)         49408     
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 5, 1, 256)         0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 1, 1, 128)         163968    
_________________________________________________________________
flatten_17 (Flatten)         (None, 128)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 129       
Total params: 213,505
Trainable params: 213,505
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dens

c:\users\notebook2\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.326508, acc.: 0.00%] [G loss: 2.454106]
too too from too from too from 
too too from too too too from 
too too from too too too from 
too too too too from too too 
too too from too from too too 
too too from too from too from 
too too from too from too from 
too too from too from too too 
too too from too from too from 
too too from too from too too 
too too from too from too too 
too too from too too too from 
too too from too from too too 
too too from too from too too 
too too from too from too too 
too too from too from too from 
too too from too too too from 
too too from too too too too 
too too from too from too from 
too too from too too too too 
too too from too too too from 
too too from too from too too 
too too from too from too from 
too too from too too too from 
too too from too from too too 


c:\users\notebook2\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 1.445808, acc.: 0.00%] [G loss: 0.335599]
2 [D loss: 1.575082, acc.: 0.00%] [G loss: 0.339523]
3 [D loss: 1.255063, acc.: 0.00%] [G loss: 0.429417]
4 [D loss: 1.309397, acc.: 0.00%] [G loss: 0.449867]
5 [D loss: 1.355660, acc.: 0.00%] [G loss: 0.402738]
6 [D loss: 1.348910, acc.: 0.00%] [G loss: 0.386225]
7 [D loss: 1.306595, acc.: 0.00%] [G loss: 0.391359]
8 [D loss: 1.253108, acc.: 0.00%] [G loss: 0.395709]
9 [D loss: 1.252030, acc.: 0.00%] [G loss: 0.397544]
10 [D loss: 1.214879, acc.: 0.00%] [G loss: 0.391851]
11 [D loss: 1.210647, acc.: 0.00%] [G loss: 0.387123]
12 [D loss: 1.219715, acc.: 0.00%] [G loss: 0.387498]
13 [D loss: 1.189231, acc.: 0.00%] [G loss: 0.388302]
14 [D loss: 1.178473, acc.: 0.00%] [G loss: 0.388694]
15 [D loss: 1.180424, acc.: 0.00%] [G loss: 0.385494]
16 [D loss: 1.213764, acc.: 0.00%] [G loss: 0.388563]
17 [D loss: 1.158913, acc.: 0.00%] [G loss: 0.388805]
18 [D loss: 1.174827, acc.: 0.00%] [G loss: 0.384279]
19 [D loss: 1.185552, acc.: 0.00%] [G

104 [D loss: 1.331580, acc.: 0.00%] [G loss: 0.372098]
105 [D loss: 1.363930, acc.: 0.00%] [G loss: 0.370034]
106 [D loss: 1.371752, acc.: 0.00%] [G loss: 0.372988]
107 [D loss: 1.400984, acc.: 0.00%] [G loss: 0.374049]
108 [D loss: 1.383749, acc.: 0.00%] [G loss: 0.371449]
109 [D loss: 1.412013, acc.: 0.00%] [G loss: 0.371879]
110 [D loss: 1.386787, acc.: 0.00%] [G loss: 0.368448]
111 [D loss: 1.439536, acc.: 0.00%] [G loss: 0.371090]
112 [D loss: 1.386048, acc.: 0.00%] [G loss: 0.369385]
113 [D loss: 1.382264, acc.: 0.00%] [G loss: 0.363095]
114 [D loss: 1.440922, acc.: 0.00%] [G loss: 0.365020]
115 [D loss: 1.430380, acc.: 0.00%] [G loss: 0.369424]
116 [D loss: 1.424918, acc.: 0.00%] [G loss: 0.367795]
117 [D loss: 1.421468, acc.: 0.00%] [G loss: 0.366491]
118 [D loss: 1.407643, acc.: 0.00%] [G loss: 0.364175]
119 [D loss: 1.467343, acc.: 0.00%] [G loss: 0.367784]
120 [D loss: 1.412907, acc.: 0.00%] [G loss: 0.368845]
121 [D loss: 1.456857, acc.: 0.00%] [G loss: 0.369445]
122 [D los

KeyboardInterrupt: 

In [25]:
for i in range(100):
    t = GAN2vec.predict()
    print(t)

<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend someone tanaka 
<s> i 'm glad lend someone tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend someone tanaka 
<s> i 'm glad lend someone tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend someone tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend someone tanaka 
<s> i 'm glad lend someone tanaka 
<s> i 'm glad lend someone tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend dance tanaka 
<s> i 'm glad lend someone 